<a href="https://colab.research.google.com/github/grnbatera/TFC/blob/main/TFC_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [88]:
!git clone https://github.com/grnbatera/TFC.git

fatal: destination path 'TFC' already exists and is not an empty directory.


In [89]:
#Instalando módulo i/o do tensorflow
!pip install -q tensorflow-io

In [91]:
#importando bibliotecas
%matplotlib inline  
import matplotlib.pyplot as plt #importa o pacote de plotagem
import numpy as np #pacote numpy faz manipulações numéricas
from IPython.display import Audio #importando ferramentas de audio
import librosa #importando biblioteca de ferramentas amplas de manipulação de audio
import librosa.display #importando ferramentas de spectograma do librosa
import librosa.feature
import pandas as pd
import math
import seaborn as sns
import matplotlib as mpl
import tensorflow as tf
import tensorflow_io as tfio
import matlab
import scipy
from scipy import signal

In [92]:
mpl.rcParams['font.family'] = 'Roboto'
plt.rcParams['font.family'] = 'Roboto'
sns.set(font='Roboto',
        rc={
 'axes.axisbelow': False,
 'axes.edgecolor': 'lightgrey',
 'axes.facecolor': 'None',
 'axes.grid': False,
 'axes.labelcolor': 'dimgrey',
 'axes.spines.right': False,
 'axes.spines.top': False,
 'figure.facecolor': 'white',
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'text.color': 'black',
 'xtick.bottom': False,
 'xtick.color': 'dimgrey',
 'xtick.direction': 'out',
 'xtick.top': False,
 'ytick.color': 'dimgrey',
 'ytick.direction': 'out',
 'ytick.left': False,
 'ytick.right': False})
 
sns.set_context("notebook", rc={"font.size":20,
                                "axes.titlesize":24,
                                "axes.labelsize":18})
                             

In [115]:
a1 = [1, -0.3];
g = .995;
b1 = g * (a1[0]+a1[1])
b1


0.6965

In [94]:
fname1='/content/TFC/body1.wav'
#pegando os valores de sr e atribuindo audio para y
y1, fs=librosa.load(fname1, sr=None, mono=True) #lendo o fname, se sr diferente de None ele reamostra pra 22050 Hz

In [95]:
y1

array([-0.00189209,  0.00064087, -0.00296021, ..., -0.00219727,
       -0.00170898, -0.0015564 ], dtype=float32)

In [96]:
y1
Audio(data=y1,rate=fs)

In [97]:
fs

22050

In [98]:
f=440
phi=0.4275
D = math.floor(fs/f - phi);
d = fs/f - phi - D;

if d < 0.1:
  d = d + 1;
  D = D - 1;

In [99]:
D

49

In [100]:
d

0.6861363636363649

In [101]:
eta = (1-d) / (1+d);
b2 = [eta,1];
a2 = [1,eta];


In [102]:

num=np.convolve(b1,b2)
den=np.convolve(a1,a2)



In [104]:
den

array([ 1.        , -0.11385631, -0.05584311])

In [105]:
num

array([0.12964908, 0.6965    ])

In [106]:
k=D-len(den)
b=den
z=-num
m=np.zeros((k,))
a=np.hstack((b,m,z))
a

array([ 1.        , -0.11385631, -0.05584311,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        , -0.12964908,
       -0.6965    ])

In [107]:
b

array([ 1.        , -0.11385631, -0.05584311])

In [108]:
DUR=3

In [109]:
z1=np.zeros((DUR*fs-len(y1)),);
z1
x=np.hstack((y1,z1))

In [114]:
y = scipy.signal.lfilter(b,a,x);

#Normalizando y 
y = y/(max(abs(y))+.1);
y

array([-1.61855619e-03,  5.48220646e-04, -2.53225727e-03, ...,
       -1.92871113e-06, -1.90665342e-06, -1.87739659e-06])

In [111]:
Audio(data=y,rate=fs)